# Time Series Anomaly Detection

We will use a concept known as **Bollinger Bands** to discover when behavior changes from what is "expected" or normal. **Bollinger Bands** is a volatility indicator and commonly used in stock market trading.

<u>Scenario</u>: Discover anomalies in number of web pages accessed in a day by a user. Is there a bot copying the curriculum?

We will accomplish this by breaking down to the following tasks:

1. Acquire the data
1. Prepare the data
1. Make the analysis process (which we will discuss later) work over all users.
1. Turn the analysis process and calculations into a function that can be used to loop through for each user.
1. Test the function on a single user.
1. Analyze by looping over all users.

The analysis process will look like this, for each user:

1. Compute necessary metrics to arrive at the final metric, %b (percent-b).
1. Add user id to the dataframe that contains all the metrics, including %b.
1. Filter to rows where %b indicates anomaly (i.e. > 1)
1. Append rows of anomalies of new user to previous users' anomalous activity.

Finally, we will do a quick sample of exploration of the anomalies. There is much more you can do!

Your exercise will be to add comments, markdown, and docstrings to the code in this lesson.

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import seaborn as sns
import env

## Acquire

After doing some research, some experimentation of performing actions and watching the logs, we discovered what each of the fields represent. We then parse and name the fields accordingly.

In [ ]:
url = f'mysql+pymysql://{env.user}:{env.password}@{env.host}/curriculum_logs'
query = '''
SELECT date,
       path as endpoint,
       user_id,
       cohort_id,
       ip as source_ip
FROM logs;
'''
df = pd.read_sql(query, url)
df.head()

## Prepare

In [ ]:
df.date = pd.to_datetime(df.date)
df = df.set_index(df.date)
pages = df['endpoint'].resample('d').count()
pages.head()

In [ ]:
pages.plot()

No need to split because we are not modeling, we are using statistics to identify low probability cases.

### Exponential Moving Average
Simple Moving Average (SMA) time series are much less noisy than the time series of the original data points. The challenge with SMA, however, is that the values of SMA lag the original values. This means that changes in the trend are only seen with a delay (lag) of L time units. For datasets that contain rapid trend shifts, a SMA may simply be too slow to be useful. 

Exponential Moving Average (EMA) helps reduce the lag induced by the use of the SMA. It does this by putting more weight on more recent observations, while the SMA weights all observations equally.

The EMA function looks like this:

### EMA<sub>*t*</sub> = α ∗ (*t*<sub>0</sub> − EMA<sub>*t−1*</sub>) + EMA<sub>*t−1*</sub>

Where:

- t<sub>0</sub> = Latest value
- t<sub>−1</sub> = Previous value
- EMA<sub>*t−1*</sub> = Exponential moving average of previous day.
- The multiplier: α = 2/(M+1)

and 

- M = Number of time periods, span of the window

However, we will use the pandas `ewm()` (Exponential Weighted functions) to compute our EMA. So we just need to define the following:

M = `span` argument = number of time periods. We will try 7 days, 14 days, and 30 days.

In [ ]:
# 7 day EMA
ema_7d = pages.ewm(span=7).mean()
ema_7d.head()

Notice how there are no missing values. `ewm()` will use as many values as are available to compute the mean. 

So if the span is 7 days, but there is only one day of data available (i.e. the first day), the EMA will equal the first value.

In [ ]:
ema_7d[0] == pages[0]

In [ ]:
pages.head()

In [ ]:
# 14 days EMA
ema_14d = pages.ewm(span=14).mean()
ema_14d.head()

In [ ]:
# 30 days EMA
ema_30d = pages.ewm(span=30).mean()
ema_30d.head()

In [ ]:
# 90 days EMA
ema_90d = pages.ewm(span=90).mean()
ema_90d.head()

> Remember! These are *exponential* moving averages, not simple!

In [ ]:
fig, ax = plt.subplots(figsize=(16,8))

ax.plot(pages.index, pages, label='Daily', alpha=.5)

ax.plot(pages.index, ema_7d, label = '7-day EMA')
ax.plot(pages.index, ema_14d, label = '14-day EMA')
ax.plot(pages.index, ema_30d, label = '30-day EMA')
ax.plot(pages.index, ema_90d, label = '90-day EMA')

ax.legend(loc='best')
ax.set_ylabel('Number of pages')

plt.show()

Let's look at a smaller date range to see how these EMAs react to changes in value:

In [ ]:
fig, ax = plt.subplots(figsize=(16,8))

ax.plot(pages.index[:100], pages[:100], label='Daily', alpha=.5)
ax.plot(pages.index[:100], ema_7d[:100], label = '7-day EMA')
ax.plot(pages.index[:100], ema_14d[:100], label = '14-day EMA')
ax.plot(pages.index[:100], ema_30d[:100], label = '30-day EMA')
ax.plot(pages.index[:100], ema_90d[:100], label = '90-day EMA')

ax.legend(loc='best')
ax.set_ylabel('Number of pages')

plt.show()

## Bollinger Bands

- A volatility indicator commonly used in stock market, forex, and cryptocurrency trading.
- Made up of 3 lines, the Upper Band (UB), the Lower Band (LB) and the Midband.
- Based on the exponential moving average

### Midband

The Exponential Moving Average

`midband = train.ewm(span=30).mean()`

### Upper & Lower Band

UB/LB = Midband +/- stdev * K

`stdev = train.ewm(span=30).std()`

K = the number of standard deviations to go up and down from the EMA

### %b, Percent Bandwidth

Shows where the last value sits in relation to the bands.

%b = last−LB / UB-LB

- If %b > 1, data point lies above the upper band
- If %b < 0, data point lies below the lower band
- If %b == .5, data point lies on the midband.

### Bandwidth

The width of the bands

Bandwidth = (UB-LB) / midband

In [ ]:
# set the window span
span = 30

# compute midband
midband = pages.ewm(span=span).mean()

midband.head()

In [ ]:
# compute exponential stdev
stdev = pages.ewm(span=span).std()

stdev.head()


In [ ]:
# compute upper and lower bands
ub = midband + stdev*3
lb = midband - stdev*3
ub, lb

In [ ]:
# concatenate ub and lb together into one df, bb
bb = pd.concat([ub, lb], axis=1)

bb.columns = ['ub', 'lb']
bb.head()

In [ ]:
my_df = pd.concat([pages, midband, bb], axis=1)
my_df.columns = ['pages', 'midband', 'ub', 'lb']
my_df.head()

### Plot the bands

In [ ]:
fig, ax = plt.subplots(figsize=(12,8))

ax.plot(my_df.index, my_df.pages, label='Number of Pages')

ax.plot(my_df.index, my_df.midband, label = '30-day EMA/midband')
ax.plot(my_df.index, my_df.ub, label = 'Upper Band')
ax.plot(my_df.index, my_df.lb, label = 'Lower Band')

ax.legend(loc='best')
ax.set_ylabel('Number of pages')

plt.show()

### Compute %b

Each data point will have a %b value that represents its relative position within/around the bollinger bands. It answers the question: Where does this point sit relative to a simple range of expected values? For a trader, its usefulness is based on the validity of an underlying assumption that the overall trend will remain constant. If this is the case, then subsequent points should regress back towards the moving average and can be used in high frequency trading setups (i.e. "scalping"). 

Lets manually compute %b for the last value in the dataset:

In [ ]:
# Visualize the last record in our dataset
my_df.iloc[-1]

In [ ]:
# get the value of the last record
last_measure = my_df.iloc[-1].pages
last_measure

In [ ]:
# get the lower band value on the last date
last_lb = my_df.iloc[-1].lb
last_lb

In [ ]:
# get the upper band value on the last date
last_ub = my_df.iloc[-1].ub
last_ub

In [ ]:
# compute %b
last_pct_b = (last_measure - last_lb)/(last_ub - last_lb)

print('%b for last datapoint: ', round(last_pct_b, 2))

In [ ]:
my_df.tail(3)

We can add a new column that shows the %b for every record in the dataset (except the first because there are no bollinger bands available for that one record)

In [ ]:
my_df['pct_b'] = (my_df['pages'] - my_df['lb'])/(my_df['ub'] - my_df['lb'])

my_df.head()

Now, where do we see anomalies? We will search for %b values > 1. In this specific example, we don't need to search for values < 0 because a low extreme is not something we are concerned about. 

In [ ]:
my_df[my_df['pct_b']>1]

No anomalies found. Remember, the cutoff is based on the arbitrary selection for K earlier. We could always pick a different K.

Anyways great! There are no anomalies! Problem solved. 

...not so fast.

During our preparation step we resampled the data to a daily period. The bollinger bands and %b are only looking for individual days where the overall aggregate log activity was extreme. If the number of users is large enough, then an individual who is scraping our data may not be extreme enough to push a single day that far from the EMA. 

We need to dig deeper. Lets look at our data on a user by user basis:

In [ ]:
def acquire_logs(user=env.user, password=env.password, host=env.host):
    '''
    This function queries the Codeup MySQL curriculum_logs database and returns a dataframe
    '''
    url = f'mysql+pymysql://{env.user}:{env.password}@{env.host}/curriculum_logs'
    query = '''
    SELECT date,
           path as endpoint,
           user_id,
           cohort_id,
           ip as source_ip
    FROM logs;
    '''
    df = pd.read_sql(query, url)
    return df

def one_user_df_prep(df, user):
    '''
    This function returns a dataframe consisting of data for only a single defined user
    '''
    df = df[df.user_id == user]
    df.date = pd.to_datetime(df.date)
    df = df.set_index(df.date)
    pages_one_user = df['endpoint'].resample('d').count()
    return pages_one_user

def compute_pct_b(pages_one_user, span, weight, user):
    '''
    This function adds the %b of a bollinger band range for the page views of a single user's log activity
    '''
    # Calculate upper and lower bollinger band
    midband = pages_one_user.ewm(span=span).mean()
    stdev = pages_one_user.ewm(span=span).std()
    ub = midband + stdev*weight
    lb = midband - stdev*weight
    
    # Add upper and lower band values to dataframe
    bb = pd.concat([ub, lb], axis=1)
    
    # Combine all data into a single dataframe
    my_df = pd.concat([pages_one_user, midband, bb], axis=1)
    my_df.columns = ['pages_one_user', 'midband', 'ub', 'lb']
    
    # Calculate percent b and relevant user id to dataframe
    my_df['pct_b'] = (my_df['pages_one_user'] - my_df['lb'])/(my_df['ub'] - my_df['lb'])
    my_df['user_id'] = user
    return my_df

def plot_bands(my_df, user):
    '''
    This function plots the bolliger bands of the page views for a single user
    '''
    fig, ax = plt.subplots(figsize=(12,8))
    ax.plot(my_df.index, my_df.pages_one_user, label='Number of Pages, User: '+str(user))
    ax.plot(my_df.index, my_df.midband, label = 'EMA/midband')
    ax.plot(my_df.index, my_df.ub, label = 'Upper Band')
    ax.plot(my_df.index, my_df.lb, label = 'Lower Band')
    ax.legend(loc='best')
    ax.set_ylabel('Number of Pages')
    plt.show()

def find_anomalies(df, user, span, weight, plot=False):
    '''
    This function returns the records where a user's daily activity exceeded the upper limit of a bollinger band range
    '''
    
    # Reduce dataframe to represent a single user
    pages_one_user = one_user_df_prep(df, user)
    
    # Add bollinger band data to dataframe
    my_df = compute_pct_b(pages_one_user, span, weight, user)
    
    # Plot data if requested (plot=True)
    if plot:
        plot_bands(my_df, user)
    
    # Return only records that sit outside of bollinger band upper limit
    return my_df[my_df.pct_b>1]

# Using our functions:

In [ ]:
df = acquire_logs()
df.head()

#### Test function on a single user:

In [ ]:
user = 1
span = 30
weight = 6
user_df = find_anomalies(df, user, span, weight)

anomalies = pd.DataFrame()
user_df = find_anomalies(df, user, span, weight)
anomalies = pd.concat([anomalies, user_df], axis=0)

In [ ]:
anomalies.head()

No daily anomalies for user 1

#### Use function in a loop examining all users:

In [ ]:
span = 30
weight = 3.5

anomalies = pd.DataFrame()
for u in list(df.user_id.unique()):
    user_df = find_anomalies(df, u, span, weight)
    anomalies = pd.concat([anomalies, user_df], axis=0)

In [ ]:
anomalies

What if we sort by %b?

In [ ]:
anomalies.sort_values(by='pct_b', ascending=False)

Not super useful. Looks like there could be a fair number of "false positives" here. Lets sort by the page counts:

In [ ]:
anomalies.sort_values(by='pages_one_user', ascending=False)

#### Wow! User_id #341 looked at 272 pages in a single day. Lets plot their activity:

In [ ]:
df_341 = one_user_df_prep(df, 341)

In [ ]:
df_341.plot(figsize=(16,8))

Are there users that show up more frequently than others?

In [ ]:
anomalies.user_id.value_counts()

Let's plot user #88:

In [ ]:
df_88 = one_user_df_prep(df, 88)
df_88.plot(figsize=(16,8))

# SUSPICIOUSSSSS... {ಠʖಠ}

# Exercises
- file name: `time_series_anomaly_detection.py` or `time_series_anomaly_detection.ipynb`

The dataset for these exercises lives in the Codeup Data Science MySQL Server. The database name is curriculum_logs.

**Hint**: You will need to explore the database and significantly adjust your acquisition step (yay SQL!)

1. Label students by the program they are in.
1. Is it possible to identify the user_id of a staff member?
1. Identify students who are accessing our curriculum pages beyond the end of their time at Codeup.
1. Identify students who present anomalous activity using the Bollinger Band method, but reduce K to 2. 
1. Plot the access activity of these students.

**BONUS:** Identify users who are viewing both the web dev and data science curriculum 